In [10]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
import requests
import re
import time
from tqdm import tqdm
import json
import pprint

In [ ]:
with open("../data/鉄道人身事故データベース.html", "r") as f:
    html = f.read()
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.findAll("table")[0].findAll("tr")
    with open("../data/jinshin.csv", "w", newline="", encoding="utf-8") as file:
        csv_write = csv.writer(file)
        for tr in a:
            data = []
            for cell in tr.findAll(["td", "th"]):
                if (not "詳細" in cell.get_text()) and cell.get_text() != "":
                    data.append(cell.get_text())
            if len(data) == 3:
                data.append("")
            for url in tr.findAll("a"):
                data.append(url.get("href"))
            csv_write.writerow(data)

In [ ]:
df = pd.read_csv("../data/jinshin.csv", header=0, names=["date", "line", "place", "injury", "url"])
data = []
sentences = []
for url in tqdm(df["url"]):
    d = BeautifulSoup(requests.get(url).text,'html.parser')
    a = d.findAll("a",text=re.compile("魚拓"))
    if a :
        data.append(a[0].get("href"))
    else :
        data.append("")
    sentence = d.findAll("p",class_="card-text")
    if sentence:
        sentences.append(sentence[0].get_text().replace(" ","").replace("\n","").replace("　",""))
    else :
        sentences.append("")
    time.sleep(0.01)
# 保存
save_data = np.array(data)
save_sentences = np.array(sentences)
np.save("archive_url3",save_data)
np.save("news_sentences", save_sentences)
# 付け加えて保存
df["archive"] = data
df["sentence"] = sentences
df.to_csv("../data/jinshin_archive.csv")

In [ ]:
url = df["url"][0]
d = BeautifulSoup(requests.get(url).text, 'html.parser')

gyotaku = d.findAll("a", text =re.compile("魚拓"))[0].get("href")
print(gyotaku)
print(d.findAll('p',class_="card-text")[0].get_text())

In [9]:
df = pd.read_csv("../data/jinshin_archive.csv", header=0, names=["date", "line", "place", "injury", "url", "archive", "sentence"])
data = []
sentences = []
for url in tqdm(df["url"]):
    d = BeautifulSoup(requests.get(url).text,'html.parser')
    sentence = d.findAll("p",class_="card-text")
    if sentence:
        sentences.append(sentence[0].get_text().replace(" ","").replace("\n","").replace("　",""))
    else :
        sentences.append("")
    time.sleep(0.01)
# 保存
save_sentences = np.array(sentences)
np.save("news_sentences", save_sentences)
# 付け加えて保存
df["sentence"] = sentences
df.to_csv("../data/jinshin_archive_sentence.csv")

100%|██████████| 14897/14897 [1:01:16<00:00,  4.05it/s]


In [29]:
with open("../data/JR東/東日本旅客鉄道東海道線.geojson") as f:
    df = json.load(f)
print(df["features"][0]["properties"]["name"])

東京


In [46]:
df = pd.read_csv("../data/jinshin_archive_sentence.csv",index_col=0)
hoge = df.place=="新大阪駅"
df[hoge]

,date,line,place,injury,url,archive,sentence
3350,2019/06/18 09:58,JR京都線,新大阪駅,男性/死亡,https://jinshinjiko.com/accidents/11544,http://archive.is/fNXE8,6月18日朝、大阪市淀川区西中島のJR新大阪駅において、男性と電車が接触し、同男性は搬送先の...
7461,2015/10/29 22:59,東海道線_(JR西日本),新大阪駅,男性/軽傷,https://jinshinjiko.com/accidents/1076,NaN,10月29日夜、大阪市淀川区西中島のＪＲ京都線新大阪駅において、男性が電車と接触し、怪我を負...
9305,2014/03/25 21:20,大阪市営地下鉄御堂筋線,新大阪駅,34歳/女性/重傷,https://jinshinjiko.com/accidents/2143,https://archive.is/jKuee,２５日午後９時２０分ごろ、大阪市淀川区西中島の市営地下鉄御堂筋線新大阪駅で、ホームから線路に...
10580,2013/03/31 06:05,大阪市営地下鉄御堂筋線,新大阪駅,NaN,https://jinshinjiko.com/accidents/3486,NaN,NaN
11365,2012/09/01 12:03,大阪市営地下鉄御堂筋線,新大阪駅,30代/女性/軽傷,https://jinshinjiko.com/accidents/4078,http://b.hatena.ne.jp/entry/mainichi.jp/select...,１日午後０時５分ごろ、大阪市淀川区西中島５の市営地下鉄御堂筋線新大阪駅で、天王寺発新大阪行き...
12465,2011/10/22 18:27,東海道線_(JR西日本),新大阪駅,33歳/男性/死亡,https://jinshinjiko.com/accidents/5324,http://b.hatena.ne.jp/entry/sankei.jp.msn.com/...,２２日午後６時半ごろ、大阪市淀川区西中島のＪＲ新大阪駅で、京都発西明石行きの下り普通電車の運...
12870,2011/06/29 19:51,東海道線_(JR西日本),新大阪駅,NaN,https://jinshinjiko.com/accidents/5792,NaN,NaN
